In [105]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 設定 data_path
dir_data = './Downloads/'

In [106]:
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

(307511, 122)

In [107]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,M,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,F,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,M,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,F,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,M,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [108]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])

In [109]:
app_train['AMT_INCOME_TOTAL'].max()

117000000.0

In [110]:
app_train['AMT_INCOME_TOTAL'].min()

25650.0

In [111]:
app_train['AMT_INCOME_TOTAL_mil'] = app_train['AMT_INCOME_TOTAL']/100000
app_train['AMT_INCOME_TOTAL_mil'].values

array([2.025, 2.7  , 0.675, ..., 1.53 , 1.71 , 1.575])

In [112]:
bin_cut = [0,1,2,3,4,5,6,7,8,9,10,np.inf]
app_train['AMT_INCOME_TOTAL_MILBIN'] = pd.cut(app_train['AMT_INCOME_TOTAL_mil'],bins=bin_cut)
print(app_train['AMT_INCOME_TOTAL_MILBIN'].value_counts())


(1.0, 2.0]     155898
(2.0, 3.0]      65176
(0.0, 1.0]      63698
(3.0, 4.0]      14676
(4.0, 5.0]       5361
(5.0, 6.0]       1095
(6.0, 7.0]        869
(8.0, 9.0]        298
(10.0, inf]       250
(7.0, 8.0]        162
(9.0, 10.0]        28
Name: AMT_INCOME_TOTAL_MILBIN, dtype: int64


In [113]:
app_train['AMT_INCOME_TOTAL_MILBINq'] = pd.qcut(app_train['AMT_INCOME_TOTAL_mil'],10)
app_train['AMT_INCOME_TOTAL_MILBINq'].value_counts()

(1.125, 1.35]    48849
(1.8, 2.25]      44809
(0.99, 1.125]    36907
(0.255, 0.81]    33391
(1.472, 1.62]    31120
(1.62, 1.8]      30704
(0.81, 0.99]     30280
(2.7, 1170.0]    27161
(2.25, 2.7]      19957
(1.35, 1.472]     4333
Name: AMT_INCOME_TOTAL_MILBINq, dtype: int64

In [114]:

INCOME=app_train[['DAYS_BIRTH','AMT_INCOME_TOTAL']]
INCOME['YEARS_BIRTH'] = app_train['DAYS_BIRTH'] / 365
INCOME['AMT_INCOME_TOTAL_mil'] = app_train['AMT_INCOME_TOTAL']/100000
bin_cut = [0,1,2,3,4,5,6,7,8,9,10,np.inf]
INCOME['AMT_INCOME_TOTAL_MILBIN'] = pd.cut(app_train['AMT_INCOME_TOTAL_mil'],bins=bin_cut)
INCOME['AMT_INCOME_TOTAL_MILBINq'] = pd.qcut(app_train['AMT_INCOME_TOTAL_mil'],10)
INCOME

C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Admin\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

,DAYS_BIRTH,AMT_INCOME_TOTAL,YEARS_BIRTH,AMT_INCOME_TOTAL_mil,AMT_INCOME_TOTAL_MILBIN,AMT_INCOME_TOTAL_MILBINq
0,9461,202500.0,25.920548,2.025,"(2.0, 3.0]","(1.8, 2.25]"
1,16765,270000.0,45.931507,2.700,"(2.0, 3.0]","(2.25, 2.7]"
2,19046,67500.0,52.180822,0.675,"(0.0, 1.0]","(0.255, 0.81]"
3,19005,135000.0,52.068493,1.350,"(1.0, 2.0]","(1.125, 1.35]"
4,19932,121500.0,54.608219,1.215,"(1.0, 2.0]","(1.125, 1.35]"
...,...,...,...,...,...,...
307506,9327,157500.0,25.553425,1.575,"(1.0, 2.0]","(1.472, 1.62]"
307507,20775,72000.0,56.917808,0.720,"(0.0, 1.0]","(0.255, 0.81]"
307508,14966,153000.0,41.002740,1.530,"(1.0, 2.0]","(1.472, 1.62]"
307509,11961,171000.0,32.769863,1.710,"(1.0, 2.0]","(1.62, 1.8]"
